# Libraries

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import pickle
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import requests
import random
from bs4 import BeautifulSoup

###### Calling spotipy

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope = "playlist-read-private",
                                               client_id="3b823a01133645dc88af2d847c9de4f1",
                                               client_secret="f6d0ddc96e6146ce8979bd711437efb8",
                                               redirect_uri="http://localhost:3000"))

----

# Functions

###### Normalizing

In [3]:
def normalize_numeric_data(numeric):
    numeric_min_max = numeric.copy()
    scaler = MinMaxScaler()

    for col in numeric.columns:
        numeric_min_max[col] = scaler.fit_transform(numeric[[col]])

    return numeric_min_max

###### Clustering with KMeans

In [4]:
def perform_clustering(numeric_min_max, df_combined):
    kmeans = KMeans(n_clusters=50, random_state=1234)
    kmeans.fit(numeric_min_max)
    clusters = kmeans.predict(numeric_min_max)
    final_df = df_combined.copy()
    final_df["cluster"] = clusters

    return final_df

##### Getting the recommendation song

In [5]:
def get_song_title(final_df):
    last_cluster_value = final_df["cluster"].iloc[-1]  # Get the value of the last row in the "cluster" column
    filtered_df = final_df[final_df["cluster"] == last_cluster_value]
    songs_to_choose_from = filtered_df.iloc[:-1]
    random_song = songs_to_choose_from.sample(random_state=random.seed())

    song_id = random_song["Song ID"].values[0]

    #search random id
    song_url = f"https://open.spotify.com/track/{song_id}"
    response = requests.get(song_url)
    soup = BeautifulSoup(response.text, "html.parser")
    song_title = soup.title.get_text()

    return song_title

###### Search for that song and returning a recommendation

In [6]:
def search_song(df):
    # User input to search for a song
    search_query = input("Enter a Rock song name to search: ")

    results = sp.search(q=search_query, type='track')

    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
    else:
        print("Song not found.")
        return

    if track_id in df["Song ID"].values:
        print("It is a HOT song from our dataframe! Keep on Rocking.")
    else:
        print("It is not a HOT song from our dataframe!")

        df_newsong_id = pd.DataFrame({"Song ID": [track_id]})

        # getting the new audio features for the new track
        audio_features = sp.audio_features(track_id)[0]
        df_newsong_features = pd.DataFrame(audio_features, index=[0])

        df_newsong = pd.concat([df_newsong_id, df_newsong_features], axis=1)

        df_combined = pd.concat([df, df_newsong], axis=0).reset_index(drop=True)
        numeric = df_combined.select_dtypes(include=[np.number])
        
        #calling normalizating function
        numeric_min_max=normalize_numeric_data(numeric)
        
        #calling clustering function
        final_df=perform_clustering(numeric_min_max, df_combined)
        
        #calling geting song title
        song_title=get_song_title(final_df)
        
        print("We recommend the following song: ",song_title)

-----

------

# Clean - Music Recommendation

In [8]:
df=pd.read_csv("df.csv")
search_song(df)

Enter a Rock song name to search: You got the Love
It is not a HOT song from our dataframe!
We recommend the following song:  Nothing Compares 2 U - song and lyrics by Prince | Spotify


----